In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import numpy as np

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import os

from geomloss import SamplesLoss

from imputers import OTimputer, RRimputer

from data_scaler import DataScaler

from utils import *
from data_loaders import dataset_loader
from softimpute import softimpute, cv_softimpute

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 读取 CSV 文件
df = pd.read_csv("juhe.csv")
df

,0,1,2,3,4,5,6,7,8,9
0,95.0,NaN,4.0,92.12967,101.0,NaN,800.0,10.0,NaN,NaN
1,95.0,NaN,4.0,92.12967,101.0,NaN,800.0,10.0,NaN,NaN
2,95.0,NaN,4.0,92.12967,101.0,NaN,800.0,10.0,NaN,NaN
3,95.0,NaN,4.0,92.12967,101.0,NaN,800.0,10.0,NaN,NaN
4,95.0,NaN,4.0,92.12967,101.0,NaN,800.0,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
432,92.0,4.0,4.0,92.56840,104.0,745.0,15.0,NaN,NaN,NaN
433,85.0,10.0,5.0,73.75000,105.0,580.0,5.0,800.0,NaN,12.0
434,85.0,10.0,5.0,73.75000,105.0,580.0,5.0,800.0,NaN,12.0
435,85.0,10.0,5.0,73.75000,105.0,580.0,5.0,800.0,NaN,12.0


In [ ]:
mean_values = df.mean(skipna=True)
std_values = df.std(skipna=True)

(10,)


## scaler and thinking of the miss data

In [7]:
from sklearn.preprocessing import scale

# 原始数据


# 标记 NaN 位置
nan_mask = df.isna()

# 临时填充 NaN（例如用均值）
df_filled = df.fillna(df.mean())

# 标准化
scaled_values = scale(df_filled)

# 恢复 NaN
scaled_df = pd.DataFrame(scaled_values, columns=df.columns, index=df.index)
scaled_df[nan_mask] = np.nan

print(scaled_df)

            0         1         2         3         4         5         6  \
0    0.597689       NaN -0.514582 -0.064419  0.536007       NaN  1.177353   
1    0.597689       NaN -0.514582 -0.064419  0.536007       NaN  1.177353   
2    0.597689       NaN -0.514582 -0.064419  0.536007       NaN  1.177353   
3    0.597689       NaN -0.514582 -0.064419  0.536007       NaN  1.177353   
4    0.597689       NaN -0.514582 -0.064419  0.536007       NaN  1.177353   
..        ...       ...       ...       ...       ...       ...       ...   
432  0.395250 -0.362693 -0.514582 -0.064419  0.609558  2.224273 -2.803066   
433 -0.077109  0.278194 -0.316504 -0.064425  0.634075 -1.468859 -2.853772   
434 -0.077109  0.278194 -0.316504 -0.064425  0.634075 -1.468859 -2.853772   
435 -0.077109  0.278194 -0.316504 -0.064425  0.634075 -1.468859 -2.853772   
436 -0.077109  0.278194 -0.316504 -0.064425  0.634075 -1.468859 -2.853772   

            7   8         9  
0   -0.240850 NaN       NaN  
1   -0.240850 N

In [ ]:
# from data_scaler import DataScaler
# import numpy as np
# import pandas as pd

# # 计算每列的均值和标准差时，忽略缺失值
# data = df.to_numpy().astype(float)
# #df_data = pd.DataFrame(data)  # 将numpy数组转换为DataFrame

# scaler = DataScaler()
# mask = np.isnan(data)
# complete_data = data[~np.any(mask, axis=1)]  # 选择没有缺失值的行
# # 修改这行代码
# scaler.fit(complete_data)  # 先fit
# Mean, Std = scaler.mean_, scaler.std_  # 然后获取均值和标准差
# # 然后对整个数据集进行转换
# data_for_im_np = scaler.transform(data)






# # 然后对整个数据集进行转换
# data_for_im_np = scaler.transform(data)



# data_for_im = torch.from_numpy(data_for_im_np)
# #X_true = torch.from_numpy(ground_truth)

In [10]:
data_for_im = torch.from_numpy(scaled_df.to_numpy())
n, d = data_for_im.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
epsilon = pick_epsilon(data_for_im) #

In [11]:
sk_imputer = OTimputer(eps=epsilon, batchsize=batchsize, lr=lr, niter=2000)

In [12]:
sk_imp = sk_imputer.fit_transform(data_for_im, verbose=True, report_interval=500)

INFO:root:batchsize = 128, epsilon = 0.2398
INFO:root:Iteration 0:	 Loss: 2.7866
INFO:root:Iteration 500:	 Loss: 1.2073
INFO:root:Iteration 1000:	 Loss: 1.9491
INFO:root:Iteration 1500:	 Loss: 2.0071


## Coming Back

In [ ]:
imp = sk_imp.detach().cpu().numpy()
df_imp = pd.DataFrame(imp)   # 先detach再转换为numpy数组

# 保存为CSV文件
df_imp.to_csv('imputed_data2.csv', index=False)

In [15]:
imp = sk_imp.detach().cpu().numpy()

# 使用手动计算的均值和标准差进行逆变换
# 公式: original_value = scaled_value * std + mean
imp_original = np.zeros_like(imp)
for i in range(imp.shape[1]):
    imp_original[:, i] = imp[:, i] * std_values[i] + mean_values[i]

df_imp = pd.DataFrame(imp_original)   # 转换为DataFrame

# 保存为CSV文件
df_imp.to_csv('imputed_data1.csv', index=False)